In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [22]:
import pickle
from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance
from tqdm import tqdm
from pathlib import Path
import numpy as np
import pandas as pd
import math

In [11]:
fold_1 = pickle.load(open(str("inference_results_Equine-Fold-1.pickle"),'rb'))
fold_2 = pickle.load(open(str("inference_results_Equine-Fold-2.pickle"),'rb'))
fold_3 = pickle.load(open(str("inference_results_Equine-Fold-3.pickle"),'rb'))

In [12]:
folds = {**fold_1, **fold_2, **fold_3}

In [14]:
annotations_path = Path("../../Statistics/EIPH_Annotations_Equine.pkl")
annotations = pd.read_pickle(annotations_path)

annotations.head()

,id,image_id,image_set,image_name,grade,vector,unique_identifier,user_id,deleted
0,1433999,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 32197, 'x2': 32338, 'y1': 15975, 'y2': ...",cafc4aa3-324f-4015-9451-ad7e5bee53f3,1,False
1,1434000,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 23069, 'x2': 23180, 'y1': 11514, 'y2': ...",979ef20e-6228-4cb3-ac9a-aecea6ad75c1,1,False
2,1434001,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 9045, 'x2': 9148, 'y1': 9711, 'y2': 9814}",ef87d780-5e2e-413e-a2ff-0712f6aa1532,1,False
3,1434002,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 10538, 'x2': 10645, 'y1': 10306, 'y2': ...",0f141dc4-5589-4e5a-a504-d77e7ad94683,1,False
4,1434003,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 26013, 'x2': 26114, 'y1': 20873, 'y2': ...",f56de1f1-83ff-48d6-b67a-bfc1ea4e1349,1,False


## filter miss shaped annotations from folds

In [13]:
for file in folds:
    
    boxes = np.array(folds[file])
    scales = ((boxes[:, 2] - boxes[:, 0]) / (boxes[:, 3] - boxes[:, 1]))
    boxes = boxes[(scales < 1.1) & (scales > 0.9)]
    
    folds[file] = boxes

## find optionmal thresholds

In [24]:
def get_grade(gt, prediction):
    
    gt_mean = float(gt["grade"].mean())
    pred_mean = float(prediction[:, 4].mean())
    diff = abs(gt_mean-pred_mean)
    
    
    return gt_mean, pred_mean, diff 

In [38]:
nms_threshs = [40] 
threshs = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]

results = []
for file in tqdm(folds):
    
    dfImage = annotations[annotations["image_name"] == file] 

    for nms_thresh in nms_threshs:
        for thresh in threshs:
            
            boxes = np.array(folds[file])
            annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
            annos = annos[annos[:, 5] > thresh]
            
            gt_mean, pred_mean, diff = get_grade(dfImage, annos)
            results.append([file, nms_thresh, thresh, gt_mean, pred_mean, diff, len(dfImage), len(annos), abs(len(dfImage) - len(annos))])
            
results = pd.DataFrame(results, columns=["image_name", "nms_thresh", "thresh", "gt", "pred", "diff", "num_gt", "num_pred", "diff_num"])
results

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [04:27<00:00,  4.69s/it]


,image_name,nms_thresh,thresh,gt,pred,diff,num_gt,num_pred,diff_num
0,18_EIPH_574882 Turnbull blue-002.svs,40,0.35,0.748440,0.584321,0.164119,6090,8942,2852
1,18_EIPH_574882 Turnbull blue-002.svs,40,0.40,0.748440,0.593678,0.154762,6090,8700,2610
2,18_EIPH_574882 Turnbull blue-002.svs,40,0.45,0.748440,0.602633,0.145808,6090,8433,2343
3,18_EIPH_574882 Turnbull blue-002.svs,40,0.50,0.748440,0.613692,0.134748,6090,8136,2046
4,18_EIPH_574882 Turnbull blue-002.svs,40,0.55,0.748440,0.626912,0.121528,6090,7781,1691
...,...,...,...,...,...,...,...,...,...
565,09_EIPH_563417 L Turnbull blue.svs,40,0.60,0.853352,0.630811,0.222541,716,1097,381
566,09_EIPH_563417 L Turnbull blue.svs,40,0.65,0.853352,0.664683,0.188669,716,1008,292
567,09_EIPH_563417 L Turnbull blue.svs,40,0.70,0.853352,0.711236,0.142116,716,890,174
568,09_EIPH_563417 L Turnbull blue.svs,40,0.75,0.853352,0.743375,0.109977,716,717,1


In [40]:
nms_threshs = [40] 
threshs = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]

optimal_th = []

for nms_thresh in nms_threshs:
    for thresh in threshs:
        
        dfTh = results[(results["nms_thresh"] == nms_thresh) & (results["thresh"] == thresh)]
        
        
        optimal_th.append([nms_thresh, thresh, dfTh["diff"].mean(), dfTh["diff_num"].mean()])
        
optimal_th = pd.DataFrame(optimal_th, columns=["nms_thresh", "thresh", "diff", "diff_num"])
optimal_th

,nms_thresh,thresh,diff,diff_num
0,40,0.35,0.142521,1910.070175
1,40,0.40,0.128083,1656.982456
2,40,0.45,0.113566,1412.070175
3,40,0.50,0.102021,1182.649123
4,40,0.55,0.092540,947.105263
5,40,0.60,0.082452,712.649123
6,40,0.65,0.074691,570.789474
7,40,0.70,0.072870,731.263158
8,40,0.75,0.073103,1351.508772
9,40,0.80,0.082926,2238.771930


In [45]:
dfTh = results[(results["nms_thresh"] == 40) & (results["thresh"] == 0.65)]
dfTh["diff_rel"] = (dfTh["diff_num"] / dfTh["num_gt"]) * 100

dfTh.sort_values("diff_rel", ascending=False)

d:\anaconda3\envs\eiph_wsi\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,image_name,nms_thresh,thresh,gt,pred,diff,num_gt,num_pred,diff_num,diff_rel
506,08_EIPH_574999 R Turnbull blue.svs,40,0.65,1.842541,2.113388,0.270847,362,732,370,102.209945
106,10_EIPH_563417 R Turnbull blue-002.svs,40,0.65,0.956376,0.733055,0.223321,596,959,363,60.906040
316,16_EIPH_576172 Turnbull blue-002.svs,40,0.65,1.044256,0.892446,0.151811,1062,1562,500,47.080979
76,17_EIPH_575796 Turnbull blue.svs,40,0.65,0.892157,0.703283,0.188873,4590,6609,2019,43.986928
566,09_EIPH_563417 L Turnbull blue.svs,40,0.65,0.853352,0.664683,0.188669,716,1008,292,40.782123
186,15_EIPH_568320 Turnbull blue.svs,40,0.65,1.483416,1.325444,0.157972,5879,7943,2064,35.108012
236,22_EIPH_575216 Berliner Blau.svs,40,0.65,1.248384,1.370563,0.122179,7581,4987,2594,34.217122
86,19_EIPH_566933 L Tunrbull blue.svs,40,0.65,0.684108,0.556029,0.128079,2561,3284,723,28.231160
546,16_EIPH_574882 R Berliner Blau.svs,40,0.65,1.084204,1.219101,0.134897,1722,1246,476,27.642276
206,29_EIPH_566481 L Turnbull blue.svs,40,0.65,0.447084,0.423133,0.023951,926,683,243,26.241901


In [47]:
annotations.head()

,id,image_id,image_set,image_name,grade,vector,unique_identifier,user_id,deleted
0,1433999,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 32197, 'x2': 32338, 'y1': 15975, 'y2': ...",cafc4aa3-324f-4015-9451-ad7e5bee53f3,1,False
1,1434000,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 23069, 'x2': 23180, 'y1': 11514, 'y2': ...",979ef20e-6228-4cb3-ac9a-aecea6ad75c1,1,False
2,1434001,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 9045, 'x2': 9148, 'y1': 9711, 'y2': 9814}",ef87d780-5e2e-413e-a2ff-0712f6aa1532,1,False
3,1434002,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 10538, 'x2': 10645, 'y1': 10306, 'y2': ...",0f141dc4-5589-4e5a-a504-d77e7ad94683,1,False
4,1434003,3530,246,07_EIPH_574882 Berliner Blau-001.svs,0,"{'x1': 26013, 'x2': 26114, 'y1': 20873, 'y2': ...",f56de1f1-83ff-48d6-b67a-bfc1ea4e1349,1,False


In [50]:
imageDict = {}

for image_name in annotations["image_name"].unique():
    
    imageDf = annotations[annotations["image_name"] == image_name]
    
    image_id = list(imageDf["image_id"])[0]
    image_set = list(imageDf["image_set"])[0]
     
    imageDict[image_name] = {"image_set": image_set, "image_id": image_id}

In [53]:
nms_thresh, thresh = 40, 0.65

inference = {}

result_annos = []
for file in tqdm(folds):
    
    image_id = imageDict[file]["image_id"]
    image_set = imageDict[file]["image_set"]
    
    boxes = np.array(folds[file])
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]
    
    for anno in annos:
        grade = anno[4]
        vector = {"x1":max(0,anno[0]), "y1": max(0, anno[1]), "x2": anno[2], "y2": anno[3]}
        
        result_annos.append([0, image_id, image_set, file, grade, vector, "/", 70, False])
    
    inference[file] = annos
    
pickle.dump(inference, open(str("inference_results.p"), "wb" ))

result_annos = pd.DataFrame(result_annos, columns=["id","image_id","image_set","image_name","grade","vector","unique_identifier","user_id","deleted"])
result_annos.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [00:27<00:00,  2.07it/s]


,id,image_id,image_set,image_name,grade,vector,unique_identifier,user_id,deleted
0,0,3557,247,18_EIPH_574882 Turnbull blue-002.svs,4.0,"{'x1': 196.0, 'y1': 11203.0, 'x2': 287.0, 'y2'...",/,70,False
1,0,3557,247,18_EIPH_574882 Turnbull blue-002.svs,4.0,"{'x1': 23576.0, 'y1': 33386.0, 'x2': 23663.0, ...",/,70,False
2,0,3557,247,18_EIPH_574882 Turnbull blue-002.svs,4.0,"{'x1': 20679.0, 'y1': 34076.0, 'x2': 20764.0, ...",/,70,False
3,0,3557,247,18_EIPH_574882 Turnbull blue-002.svs,4.0,"{'x1': 26530.0, 'y1': 31209.0, 'x2': 26609.0, ...",/,70,False
4,0,3557,247,18_EIPH_574882 Turnbull blue-002.svs,4.0,"{'x1': 2310.0, 'y1': 450.0, 'x2': 2392.0, 'y2'...",/,70,False


In [56]:
result_annos.to_csv("EIPH_300_Algo.csv",sep=";")